## Boundary matrix, reduction, output barcode (using a sparse matrix) - Questions 1, 2, 3, and 4

Creating simplex class and reading function

In [34]:
import numpy as np

class simplex:
    def __init__(self, line):
        self.dim, self.val, self.vert = self.parse_line(line)
    
    def parse_line(self, line):
        splited_line = line.split()
        val = float(splited_line[0])
        dim = int(splited_line[1])
        vert = [int(x) for x in splited_line[2:]]
        assert(len(vert)==dim+1)
        return dim, val, vert

def read_filtration(name):
    simplices = []
    with open(name, "r") as ins:
        for line in ins:
            simplices.append(simplex(line))
    return simplices

Reading the example filtration

In [35]:
simplices = read_filtration('Creating_filtrations/filtrationprojplane.txt')

Grouping by time

In [36]:
simplices.sort(key=lambda x: x.val)

Sorting the vertices

In [37]:
for s in simplices:
    s.vert.sort()

Creating dictionary for matrix creation

In [38]:
simplices_str =  [str(s.vert).strip('[]') for s in simplices]

d = {}
for i, s in enumerate(simplices_str):
    d[s] = i

Creating sparse matrix

In [39]:
M = len(simplices)

sparse = [[] for i in range(M)]

for i, s in enumerate(simplices):
    if s.dim>0:
        vs = list(s.vert)
        for v in vs:
            temp = list(vs)
            temp.remove(v)
            sparse[i].append(d[str(temp).strip('[]')])
            
for i in range(M):
    sparse[i].sort()

Printing sparse matrix

In [40]:
print sparse
print M

[[], [], [], [], [], [], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [1, 2], [1, 3], [1, 4], [1, 5], [2, 3], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5], [6, 8, 12], [7, 8, 15], [11, 13, 16], [15, 16, 18], [18, 19, 20], [12, 14, 19], [11, 14, 17], [6, 9, 13], [9, 10, 20], [7, 10, 17]]
31


Reducing Matrix

In [20]:
lowsparse = [float('inf')]*M
lowdict = {}

def xor(col1, col2):
    c = []
    i, j = 0, 0
    n1, n2 = len(col1), len(col2)
    while(i<n1 and j<n2):
        if(col1[i]<col2[j]):
            c.append(col1[i])
            i+=1
        elif(col1[i]>col2[j]):
            c.append(col2[j])
            j+=1
        else:
            i+=1
            j+=1
    while i<n1:
        c.append(col1[i])
        i+=1
    while j<n2:
        c.append(col2[j])
        j+=1
    return c

for j in range(M): #*O(M)
    if j%10000==0:
        print j
    if sparse[j]: 
        lowcol = sparse[j][-1]
        lowsparse[j] = lowcol
        if lowcol not in lowdict:
            lowdict[lowcol] = j
            continue
        while lowcol in lowdict: #*O(M) in the worst case
            ind = lowdict[lowcol]
            sparse[j] = xor(sparse[j],sparse[ind]) #*O(M) in the worst case too (depends on the size of the sparse columns)
            if sparse[j]:
                lowcol = sparse[j][-1]
                lowsparse[j] = lowcol
            else:
                lowsparse[j] = float("inf")
                break
        if sparse[j]:
            lowdict[lowcol] = j
    else:
        lowsparse[j]=float("inf")

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


KeyboardInterrupt: 

In [41]:
lowsparse = [float('inf')]*M
lowdict = {}
index_with_i_as_low = {}

def xor(col1, col2):
    c = []
    i, j = 0, 0
    n1, n2 = len(col1), len(col2)
    while(i<n1 and j<n2):
        if(col1[i]<col2[j]):
            c.append(col1[i])
            i+=1
        elif(col1[i]>col2[j]):
            c.append(col2[j])
            j+=1
        else:
            i+=1
            j+=1
    while i<n1:
        c.append(col1[i])
        i+=1
    while j<n2:
        c.append(col2[j])
        j+=1
    return c

for j in range(M): #*O(M)
    if j%10000==0:
        print j    
    if(len(sparse[j])==0):
        lowsparse[j] = float("inf")
    while len(sparse[j])>0: 
        lowcol = sparse[j][-1]
        lowsparse[j] = lowcol
        # while the lowest element of the column i has already appeared as the last element of a previous column
        # so the index is different from -1
        if lowcol in index_with_i_as_low:
            # if this row is the lowest row in some other column,
            # then we must subtract this other column from the current column
            ind = index_with_i_as_low[lowcol]
            sparse[j] = xor(sparse[j], sparse[ind])
            lowsparse[j] = float("inf")
        else:
            # else, we have just found the lowest element from the column
            # and can end the reduction of this column
            index_with_i_as_low[lowcol] = j
            break


0


## Explaning reduction complexity
In the code above, there are two nested loops, both of them being executed at most O(M) times. In fact, the first one is an iteration in range(M). The internal one is executed as long as sparse[j] is not empty and a break command is not called. Since the index of the lowest element in sparse[j] is strictly decreasing and it starts with some value smaller than M, this loop cannot be executed more than M times.

One can also notice that there are at most O(M) operations inside each iteration of the loop, since all operations but the xor (or subtraction) of columns take constant time, and the xor operation takes O(M) time.

Therefore, the total complexity of the algorithm is bounded by O(M*M*M) = O(M^3)

Printing reduced matrix

In [42]:
print sparse

[[], [], [], [], [], [], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [], [], [], [], [], [], [], [], [], [], [6, 8, 12], [7, 8, 15], [11, 13, 16], [15, 16, 18], [18, 19, 20], [12, 14, 19], [11, 14, 17], [6, 9, 13], [7, 8, 9, 10, 11, 12, 13, 14], []]


Giving final answer

In [43]:
finalanswer = []
for j in range(M):
    if lowsparse[j] == float('inf'):
        if j not in lowsparse:
            aux = simplices[j]
            finalanswer.append(str(aux.dim)+' '+str(aux.val)+' '+'inf')
    else:
        aux1 = simplices[lowsparse[j]]
        aux2 = simplices[j]
        finalanswer.append(str(aux1.dim)+' '+str(aux1.val)+' '+str(aux2.val))
print finalanswer

['0 1.0 inf', '0 1.0 2.0', '0 1.0 2.0', '0 1.0 2.0', '0 1.0 2.0', '0 1.0 2.0', '1 2.0 inf', '1 2.0 3.0', '1 2.0 3.0', '1 2.0 3.0', '1 2.0 3.0', '1 2.0 3.0', '1 2.0 3.0', '1 2.0 3.0', '1 2.0 3.0', '1 2.0 3.0', '2 3.0 inf']


Returning as a txt

In [31]:
def output_codebar(barcodelist,name):
    barcodetxt = open(name,"w")
    for code in barcodelist:
        barcodetxt.write(code)
        barcodetxt.write("\n")
    return

output_codebar(finalanswer,"output.txt")